In [1]:
import numpy as np 
import pandas as pd
import nltk as nl
import re
import os
from nltk.corpus import cmudict
from math import sqrt,log
from collections import Counter

In [2]:
letters = len(re.findall('[a-zA-Z0-9]',"cartoons are okay but i need some actual entertainment"))

In [3]:
letters

46

For the sake of everyone

In [5]:
num_features = 15# sumthing you can set 
num_points =10489 # you can calculate
Dataset = np.zeros([num_points, num_features])
###we need a dictionary which can say which features is at which column 
feature_dict={}
feature_dict['0'] = 'letters/wrd'
feature_dict['1'] = 'syyl/wrd'
feature_dict['2'] = 'wrd/sent'

feature_dict['3'] = 'ttr'
feature_dict['4'] = 'root_ttr'
feature_dict['5'] = 'corrected_ttr'
feature_dict['6'] = 'bilog_ttr'
feature_dict['7'] = 'uber'

feature_dict['8'] = 'noun_var'
feature_dict['9'] = 'adverb_var'
feature_dict['10'] = 'adjective_var'
feature_dict['11'] = 'verb_var'
feature_dict['12'] = 'sq_verb_var'
feature_dict['13'] = 'cor_verb_var'
feature_dict['14'] = 'lex_var'
feature_dict['15'] = 'lex_dens'





For finding number of syllabes 

In [6]:
d = cmudict.dict()

def nsyl(word):
    try:
        x = [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
        return x[0]
    except KeyError:
        #print('error')
        #if word not found in cmudict
        return syllables(word)

def syllables(word):
#referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

To find all types of Type- token ratio

In [7]:
def find_ttr(file_content):
    token = len(nl.word_tokenize(file_content))
    ttype = len(set(nl.word_tokenize(file_content)))
    ttr = ttype/token                             #Type Token ratio
    rttr = ttype/sqrt(token)                      #root TTR
    cttr = ttype/sqrt(2*token)                    # corrected TTR
    bttr = log(ttype)/log(token)                  #Bilogarithmic TTR
    if  (log(token/ttype))!=0:
        uber = ((log(ttype)**2)/(log(token/ttype)) )     #Uber Index
    else:
        uber =0
    return ttr,rttr,cttr,bttr,uber

Lexical Variations

In [8]:
def nvar(file_contents):
    ttokens = nl.word_tokenize(file_contents.lower())# total tokens 
    text = nl.Text(ttokens)        
    tags = nl.pos_tag(text)
    count = Counter(tag for word,tag in tags)
    nn = count['NN']+count['NNS']+count['NNP']+count['NNPS'] ##only for noun
    vb = count['VB']+count['VBD']+count['VBG']+count['VBN']+count['VBP']+count['VBZ'] # for verb
    adv = count['RB']+count['RBR']+count['RBS']
    adj = count['JJ']+count['JJR']+count['JJS']
    lex_item = nn+vb+adv+adj
    return nn,vb,adv,adj,count,lex_item
    

In [9]:
Loc = "/home/sp/nlpfinal/WeeBit-TextOnly/"

#just few declarations

# Here is the core

In [14]:

def readFeatures(fileData,nameList):
    featuresDict = {}
    index = 0
    avgwps = {}
    avglpw = {}
    avgspw = {}
    nv = {}
    vv1 = {}
    svv = {}
    cvv = {}
    advvv = {}
    adjvv = {}
    LD = {}
    ttr = {}
    rttr = {}
    cttr = {}
    bttr = {}
    uber = {}
    
    for page in fileData:
        
        key = nameList[index]
        index = index + 1
        syl = 0
        file_contents = page
        words = nl.word_tokenize(file_contents)
        dwords = len(words)
        vocab = len(set(words))

        letters = len(re.findall('[a-zA-Z0-9]',file_contents))
        

        dsent = len(nl.sent_tokenize(file_contents))
        #avg number of word per sentence
        avgwps[key] = dwords/dsent 
        #done
        
        #avg number of letter per word
        avglpw[key] = letters/dwords


       #avg number of syllables per word 
        for k in range(len(words)):
            syl = syl+nsyl(words[k]) 
        avgspw[key] = syl/dwords 

        # will give number of noun,verb,adverb,adjective
        nn,vb,adv,adj, v,lex_item = nvar(file_contents) 

        # noun veriations
        nv[key] = nn/vocab  

        #verb variations1
        vv1[key] = vb /vocab   

        #squared verb variations
        svv[key] = (vb**2)/vocab 

         #corrected verb variations
        cvv[key] = vb/sqrt(2*vocab) 

         #adverb variations
        advvv[key] = adv/vocab    

         #adjective variations
        adjvv[key] = adj/vocab   

         #lexical density
        LD[key] = lex_item /dwords 

        #find ttr n its family 
        ttr[key],rttr[key],cttr[key],bttr[key],uber[key] = find_ttr(file_contents) 
        #print(i,uber[i-1])
        
    featuresDict["W/S"] = avgwps
    featuresDict["letter/W"] = avglpw
    featuresDict["Syl/W"] = avgspw
    featuresDict["NV"] = nv
    featuresDict["VV1"] = vv1
    featuresDict["svv"] = svv
    featuresDict["cvv"] = cvv
    featuresDict["advvv"] = advvv
    featuresDict["adjvv"] = adjvv
    featuresDict["LD"] = LD
    featuresDict["ttr"] = ttr
    featuresDict["rttr"] = rttr
    featuresDict["cttr"] = cttr
    featuresDict["bttr"] = bttr
    featuresDict["uber"] = uber

    return featuresDict
        
        

In [13]:
def dataReader(flistName,location):
    dataList = []
    for fname in flistName:
        file = open(location+fname,"r",encoding="cp437")
        data = file.read()
        dataList.append(data)
    return dataList


# Quality code ends here 

In [15]:
#Read the Data
loc = "/home/sp/nlpfinal/WeeBit-TextOnly/"
#BitGCSE, BitKS3, WRLevel2, WRLevel3, WRLevel4

BitGCSE = loc+"BitGCSE/" # for age group 15 to 16 year
BitKS3 = loc + "BitKS3/" # for age group 11 to 13 year
WRLevel2 = loc + "WRLevel2/"
WRLevel3 = loc + "WRLevel3/"
WRLevel4 = loc + "WRLevel4/"
l1 = os.listdir(WRLevel2)
l2 = os.listdir(WRLevel3)
l3 = os.listdir(WRLevel4)
l4 = os.listdir(BitGCSE)
l5 = os.listdir(BitKS3)

l1Data = dataReader(l1,WRLevel2)
l2Data = dataReader(l2,WRLevel3)
l3Data = dataReader(l3,WRLevel4)
l4Data = dataReader(l4,BitGCSE)
l5Data = dataReader(l5,BitKS3)


lexWR2 = readFeatures(l1Data,l1)
print("1")
lexWR3 = readFeatures(l2Data,l2)
print("2")
lexWR4 = readFeatures(l3Data,l3)
print("3")
lexbitgcse = readFeatures(l4Data,l4)
print("4")
lexbitks3 = readFeatures(l5Data,l5)
print("5")


1
2
3
4
5


In [16]:
# saveObject(lexWR2,"lexWR2")
# saveObject(lexWR3,"lexWR3")
# saveObject(lexWR4,"lexWR4")
# saveObject(lexbitks3,"lexks3")
# saveObject(lexbitgcse,"lexgcse")

In [12]:
import pickle

# How to use : create a folder name "repo" in the directory
# where file is located
# obj : the object required to save
# name : the name for the host file of the data

def saveObject(obj,name): 
    pickle.dump(obj,open( "repo/"+name+".pkl", "wb" ))

def loadObject(name):
    obj = pickle.load( open( "repo/"+name+".pkl", "rb" ) )
    return obj



In [43]:
f1.keys()

dict_keys(['W/S', 'letter/W', 'Syl/W', 'NV', 'VV1', 'svv', 'cvv', 'advvv', 'adjvv', 'LD', 'ttr', 'rttr', 'cttr', 'bttr', 'uber'])

In [44]:
f1

{'LD': {'12.txt': 0.5642857142857143,
  '218.txt': 0.5371024734982333,
  '370.txt': 0.5192307692307693,
  '475.txt': 0.5502645502645502,
  '536.txt': 0.4755244755244755},
 'NV': {'12.txt': 0.4177215189873418,
  '218.txt': 0.5,
  '370.txt': 0.5238095238095238,
  '475.txt': 0.47596153846153844,
  '536.txt': 0.5367647058823529},
 'Syl/W': {'12.txt': 1.35,
  '218.txt': 1.2756183745583038,
  '370.txt': 1.445054945054945,
  '475.txt': 1.3280423280423281,
  '536.txt': 1.2727272727272727},
 'VV1': {'12.txt': 0.31645569620253167,
  '218.txt': 0.26973684210526316,
  '370.txt': 0.32275132275132273,
  '475.txt': 0.27884615384615385,
  '536.txt': 0.29411764705882354},
 'W/S': {'12.txt': 11.666666666666666,
  '218.txt': 9.129032258064516,
  '370.txt': 13.481481481481481,
  '475.txt': 11.117647058823529,
  '536.txt': 9.533333333333333},
 'adjvv': {'12.txt': 0.12658227848101267,
  '218.txt': 0.19736842105263158,
  '370.txt': 0.15343915343915343,
  '475.txt': 0.1346153846153846,
  '536.txt': 0.18382352